# Options Pricing using Monte-Carlo Simulation

In [150]:
import numpy as np 
import math
import time
import datetime as dt
from datetime import datetime
from datetime import date
import yahoo_fin
from yahoo_fin import options
from yahoo_fin import stock_info
import pandas_datareader
from pandas_datareader import data as pdr

## Practice with Yahoo_fin

In [21]:
chain = options.get_options_chain("GME", "11/24/2023")
#chain['puts']
calls = options.get_calls("GME", "11/24/2023")
#calls
dates = options.get_expiration_dates("gme")
#dates

/home/nate/development/env/lib/python3.11/site-packages/yahoo_fin/options.py:42: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(requests.get(site, headers=headers).text)
/home/nate/development/env/lib/python3.11/site-packages/yahoo_fin/options.py:42: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(requests.get(site, headers=headers).text)


## Getting Ticker and Option Type

In [139]:
ticker = input("Enter Ticker:").upper()
price = stock_info.get_live_price(f"{ticker}")
S0 = round(price, 2)
print(f"Ticker Selected: {ticker} \n", f"Price: {price}")

option = input("Call or Put?")
print(f"Option Selected: {option}")

if option == 'put' or option == 'Put':
    chain = options.get_puts(f"{ticker}")

elif option == 'call' or option == 'Call':
    chain = options.get_calls(f"{ticker}")

/home/nate/development/env/lib/python3.11/site-packages/yahoo_fin/stock_info.py:580: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return df.close[-1]


Ticker Selected: AAPL 
 Price: 191.45
Option Selected: call


/home/nate/development/env/lib/python3.11/site-packages/yahoo_fin/options.py:42: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(requests.get(site, headers=headers).text)


## Getting a Strike Price

In [48]:
print("Select a Strike Price\n", chain['Strike'])
strike = float(input("Select a Strike Price: "))
K = strike

Select a Strike Price
 0      50.0
1      75.0
2      90.0
3      95.0
4     100.0
5     105.0
6     110.0
7     115.0
8     120.0
9     125.0
10    130.0
11    135.0
12    140.0
13    145.0
14    150.0
15    155.0
16    157.5
17    160.0
18    162.5
19    165.0
20    167.5
21    170.0
22    172.5
23    175.0
24    177.5
25    180.0
26    182.5
27    185.0
28    187.5
29    190.0
30    192.5
31    195.0
32    197.5
33    200.0
34    202.5
35    205.0
36    207.5
37    210.0
38    212.5
39    215.0
40    220.0
41    225.0
42    230.0
43    235.0
44    245.0
45    250.0
46    255.0
47    265.0
Name: Strike, dtype: float64


## Obtaining Expiry Date

In [124]:
dates = options.get_expiration_dates(f"{ticker}")
print("Select an Expiry Date\n", dates)
date1 = input("Expiry Date (YYYY, MM, DD): \n")
print(f"Expiry Date: {date1}")


Select an Expiry Date
 ['November 24, 2023', 'December 1, 2023', 'December 8, 2023', 'December 15, 2023', 'December 22, 2023', 'December 29, 2023', 'January 19, 2024', 'February 16, 2024', 'March 15, 2024', 'April 19, 2024', 'June 21, 2024', 'July 19, 2024', 'September 20, 2024', 'December 20, 2024', 'January 17, 2025', 'June 20, 2025', 'September 19, 2025', 'December 19, 2025', 'January 16, 2026']
Expiry Date: 5/30/24


## Calculating Days to Expiry

In [125]:
date_str = date1
date_object = datetime.strptime(date_str, '%m/%d/%y').date()
print(type(date_object))
print(date_object)

<class 'datetime.date'>
2024-05-30


In [171]:
from datetime import date
balls = date.today()
td = date_object - balls
print(f"Days to Expiry: {td.days}")

T = td.days / 365

Days to Expiry: 192


0.5260273972602739

## Obraining the Risk-Free Rate (3 mo. T Bills)

In [170]:
r = stock_info.get_live_price("^IRX")
rf = round(r, 2)/100

print(f"Risk-Free Rate: {rf}")

Risk-Free Rate: 0.0524


/home/nate/development/env/lib/python3.11/site-packages/yahoo_fin/stock_info.py:580: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return df.close[-1]


## Calculating Vol

In [141]:
from dateutil.relativedelta import relativedelta
minusoneyear = today - relativedelta(years=1)
print(minusoneyear)

2022-11-20


In [163]:
df = stock_info.get_data(f"{ticker}", start_date= minusoneyear, end_date= today)
df['close']


2022-11-21    148.009995
2022-11-22    150.179993
2022-11-23    151.070007
2022-11-25    148.110001
2022-11-28    144.220001
                 ...    
2023-11-13    184.800003
2023-11-14    187.440002
2023-11-15    188.009995
2023-11-16    189.710007
2023-11-17    189.690002
Name: close, Length: 250, dtype: float64

In [169]:
log_returns = np.log(df.close/df.close.shift(1)).dropna()
daily_std = log_returns.std()
annualized_vol = daily_std * np.sqrt(252)
sigma = annualized_vol
sigma

0.22714355310559806

In [172]:
iterations = input("How Many Simulations Would You Like to Run?")
print(f"Simulations = {iterations}")

Simulations = 69


In [1]:
class OptionPricing:

    def __init__(self, S0, K, T, rf, sigma, iterations):
        self.S0 = S0
        self.K = K
        self.T = T
        self.rf = rf 
        self.sigma = sigma 
        self.iterations = iterations 

    def call_option_simulation(self):

        #we have 2 columns: first with 0s, second will store the payoff
        #we need the first column of 0s: payoff function is max(S-E, 0) for call option
        option_data = np.zeros([self.iterations, 2])

        #dimensions: 1-dimensional array with as many items as the iterations
        rand = np.random.normal(0, 1, [1, self.iterations])

        #equation for the S(t) stock price
        stock_price = self.S0 * np.exp(self.T * (self.rf - 0.5 * self.sigma**2) + self.sigma *np.sqrt(self.T) * rand)

        #we need S-K in order to calculate the max(S-K, 0)
        option_data[:,1] = stock_price - self.K 

        #average for the Monte-Carlo method
        #np.amax() returns the max(S-K, 0) according to the formula
        average = np.sum(np.amax(option_data, axis=1))/float(self.iterations)

        #we have to use the exp(-rT) discount factor
        return np.exp(-1.0*self.rf*self.T)*average

    def put_option_simulation(self):

        #we have 2 columns: first with 0s, second will store the payoff
        #we need the first column of 0s: payoff function is max(S-E, 0) for call option
        option_data = np.zeros([self.iterations, 2])

        #dimensions: 1-dimensional array with as many items as the iterations
        rand = np.random.normal(0, 1, [1, self.iterations])

        #equation for the S(t) stock price
        stock_price = self.S0 * np.exp(self.T * (self.rf - 0.5 * self.sigma**2) + self.sigma *np.sqrt(self.T) * rand)

        #we need S-K in order to calculate the max(K-S, 0)
        option_data[:,1] = self.K - stock_price

        #average for the Monte-Carlo method
        #np.amax() returns the max(K-S, 0) according to the formula
        average = np.sum(np.amax(option_data, axis=1))/float(self.iterations)

        #we have to use the exp(-rT) discount factor
        return np.exp(-1.0*self.rf*self.T)*average



In [2]:
if __name__ == "__main__":

    S0 = 100                #underlying stock price at t=0
    K = 100                 #strike price
    T = 1                   #expiry
    rf = 0.05               #risk-free rate
    sigma = 0.2             #volatility of underlying stock
    iterations = 10000000   #number of iterations in the monte-carlo simulation

    model = OptionsPricing(S0, K, T, rf, sigma, iterations)
    print("Monte-Carlo Call option price: ", model.call_option_simulation())
    print("Monte-Carlo Put option price: ", model.put_option_simulation())